In [10]:
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import numpy as np

# capture start time
start_time = time.time()

# clear existing log
open('log.txt', 'w').close()

df =  pd.read_excel('Testdata.xlsx') 
vendor_grouped_df = df.groupby('Vendor')
output_df = pd.DataFrame()

for vendor_name, vendor_group in vendor_grouped_df:
    geo_region_grouped_df = vendor_group.groupby('Geo Region Group')
    for geo_region_name, geo_region_group in geo_region_grouped_df:
        geo_region_group = geo_region_group.set_index('Invoice Date')
        geo_region_group = geo_region_group.sort_index()
        start_date = geo_region_group.index.min()
        end_date = geo_region_group.index.max()
        date_table = pd.DataFrame(pd.date_range(start=start_date, end=end_date), columns=['Date'])
        geo_region_group = geo_region_group.reset_index()
        merged_df = pd.merge(date_table, geo_region_group, left_on='Date', right_on='Invoice Date', how='outer')
        merged_df['Invoice total by day'] = merged_df['Invoice total by day'].fillna(0)
        merged_df.set_index('Date', inplace=True)
        if(len(geo_region_group)>50):
            
            # ignore the warning
            warnings.filterwarnings("ignore", category=ConvergenceWarning)

            # Decompose the time series
            period_hypothesized = round(len(merged_df)/3)
            model = ExponentialSmoothing(merged_df['Invoice total by day'], trend='add', seasonal='add', seasonal_periods=period_hypothesized, freq='D')
            model_fit = model.fit()

            # Forecast the next periods
            forecast = model_fit.forecast(steps=300)
            forecast[forecast <= 0] = 0.00
            # forecast.plot()
            # plt.show()
            
            # merge the forecast with the original data
            forecast_df = pd.DataFrame(forecast, columns=['Forecasted Value'])
            merged_df = pd.concat([merged_df, forecast_df], axis=1)
            merged_df = merged_df.reset_index(drop=False)
            merged_df['Vendor'] = vendor_name
            merged_df['Geo Region Group'] = geo_region_name
            merged_df['Vendor ID'] = geo_region_group['Vendor ID'].iloc[0]
            merged_df.drop(columns=['Invoice Date'], inplace=True)
            merged_df['Invoice total by day'] = merged_df['Invoice total by day'].fillna(merged_df['Forecasted Value'])
            merged_df.drop(columns=['Forecasted Value'], inplace=True)
            output_df = pd.concat([output_df, merged_df])

            # create log file .txt
            with open('log.txt', 'a') as f:
                f.write(f"Forecasted for Vendor: {vendor_name} in Geo-region:{geo_region_name}\n")
        else:
            # create log file .txt
            with open('log.txt', 'a') as f:
                f.write(f"Not enough data to forecast for Vendor: {vendor_name} in Geo-region:{geo_region_name}\n")
            # print(f"Not enough data to forecast for Vendor: {vendor_name} in Geo-region:{geo_region_name}")
    # print(output_df)
output_df.to_csv('output.csv', index=False)
# capture end time
end_time = time.time()
# calculate total time taken
print(f"Total time taken: {end_time - start_time} seconds")

Total time taken: 4.660498857498169 seconds
